In [15]:
import os
import pandas as pd
import json
from tqdm import tqdm

# импортируем библиотеку для работы с картами
# from keplergl import KeplerGl
from ParseLog import ParseLog
from connector_db_cherepovets import PostgreSQL

from db_parameters import *

In [2]:
parser = ParseLog('log/20220511T174759.log')

df = parser.transform_df()

df.head(1)

,timestamp,clid,uuid,route,vehicle_type,latitude,longitude,speed,direction,production,garange_number,reg_number,route_descr
0,2022-05-11 16:01:20,cherepovets,ak1456_2058,#7,bus,59.134502,37.923988,15.17,192,1,,е440хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n


Наименование колонок:

- `TIMESTAMP` - время в UTC в формате дата:время
- `CLID` - vologda или cherepovets
- `UUID` - уникальный идентификатор ТС в формате id_перевозчика + "_" + id_ТС_в базе_перевозчика
- `ROUTE` - "#" + номер маршрута 
- `VEHICLE_TYPE` - 'bus', 'tramway', 'minibus', 'trolleybus'
- `LATITUDE` – широта
- `LONGITUDE` – долгота
- `SPEED` - текущая скорость
- `DIRECTION` - азимут
- `PRODUCTION` - признак производственного рейса. Когда "0" - скорее всего автобус без пассажиров. В Яндекс передаются только данные с флагом "1"
- `GARANGE_NUMBER` - опционально "человеческий" идентификатор ТС для перевозчика
- `REG_NUMBER` - госномер ТС, есть не всегда
- `ROUTE_DESCR` - текстовый идентификатор траектории маршрута в одном направлении. Нет стандартного формата, можно использовать только как "хэш", как ключ для группировки точек по траекториям

In [3]:
pg = PostgreSQL(host=HOST,database=DB,login=LOGIN,password=PASSWORD)

In [4]:
%%time

vehicle_type = ['bus', 'tramway', 'minibus', 'trolleybus']

for vehicle in vehicle_type:
    df_into = df[df.vehicle_type == vehicle]
    pg.into_pg_table(
        pg_table_name=vehicle,
        df=df_into)

CPU times: user 3.5 s, sys: 119 ms, total: 3.61 s
Wall time: 11.5 s


In [13]:
list_filename = [file for file in os.listdir('log')]

In [14]:
len(list_filename)

347

In [21]:
l_f = ['log/'+ file for file in list_filename]

In [ ]:
l_f

In [19]:
list_filename[0]

'20220526T182400.log'

In [20]:
s = 'log/' + list_filename[0]
s

'log/20220526T182400.log'

In [17]:
%%time 

vehicle_type = ['bus', 'tramway', 'minibus', 'trolleybus']

for file in list_filename:
    parser_ = ParseLog(file)
    df = parser_.transform_df()
    
    for vehicle in vehicle_type:
        df_into = df[df.vehicle_type == vehicle]
        pg.into_pg_table(
            pg_table_name=vehicle,
            df=df_into
        )

FileNotFoundError: [Errno 2] No such file or directory: '20220526T182400.log'

# Создание карты

In [94]:
with open('config.json') as f:
    config = json.load(f)

_map = KeplerGl()
_map.config = config

_map.add_data(data=df_draw, name='yelyi5ow8')

_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['yelyi5ow8'], 'id': '3ynfdw2…